In [ ]:
# import and install packages

!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm
!pip install google-colab

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from google.colab import files
from google.colab import drive
import json
from PIL import Image, ImageDraw
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

# assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00


Load Images

In [ ]:
drive.mount('/content/drive')
path_coco = "/content/damage_coco"
path_robo = "/content/damage_roboflow"

if not os.path.exists(path_coco):
    os.makedirs(path_coco)
if not os.path.exists(path_robo):
    os.makedirs(path_robo)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/College/3.\ Junior/Winter\ Semester/CS\ 474/Final\ DL/Car_Data/damage_coco.zip -d /content/damage_coco
# !unzip /content/gdrive/My\ Drive/College/3.\ Junior/Winter\ Semester/CS\ 474/Final\ DL/Car_Data/damage_roboflow.zip -d /content/damage_roboflow


Archive:  /content/drive/MyDrive/College/3. Junior/Winter Semester/CS 474/Final DL/Car_Data/damage_coco.zip
  inflating: /content/damage_coco/img/1.jpg  
  inflating: /content/damage_coco/img/10.jpg  
  inflating: /content/damage_coco/img/13.jpg  
  inflating: /content/damage_coco/img/14.jpg  
  inflating: /content/damage_coco/img/15.jpg  
  inflating: /content/damage_coco/img/16.jpg  
  inflating: /content/damage_coco/img/17.jpg  
  inflating: /content/damage_coco/img/18.jpg  
  inflating: /content/damage_coco/img/19.jpg  
  inflating: /content/damage_coco/img/2.jpg  
  inflating: /content/damage_coco/img/20.jpg  
  inflating: /content/damage_coco/img/21.jpg  
  inflating: /content/damage_coco/img/22.jpg  
  inflating: /content/damage_coco/img/23.jpg  
  inflating: /content/damage_coco/img/24.jpg  
  inflating: /content/damage_coco/img/25.jpg  
  inflating: /content/damage_coco/img/26.jpg  
  inflating: /content/damage_coco/img/27.jpg  
  inflating: /content/damage_coco/img/29.jpg  
 

In [ ]:
path_coco = "/content/damage_coco/train/0"

if not os.path.exists(path_coco):
    os.makedirs(path_coco)

In [ ]:
!mv /content/damage_coco/train/*.jpg /content/damage_coco/train/0

In [ ]:
if not os.path.exists("/content/damage_coco/masks"):
    os.makedirs("/content/damage_coco/masks")

# Load the JSON data
with open('/content/damage_coco/train/COCO_train_annos.json') as f:
    data = json.load(f)

# Get the image size
width, height = data['images'][0]['width'], data['images'][0]['height']
# Create an empty mask
mask = np.zeros((height, width), dtype=np.uint8)

last_image = 0
damage_count = 0
for annotation in data['annotations']:
  if last_image == annotation['image_id']:
    damage_count += 1
  else:
    last_image = annotation['image_id']
    damage_count = 0
  box = annotation['bbox']
  mask[box[0]:box[0]+box[2]+1, box[1]:box[1]+box[3]+1] = True
  name = '/content/damage_coco/masks/mask'+ \
          str(annotation['image_id'])+'_'+str(damage_count)+'.png'
  # np.save(name, mask)

  # Convert the array to an image
  img = Image.fromarray(mask*255)
  # Save the image as a PNG file
  img.save(name)


# # Draw polygons on the mask for each damaged area
# for annotation in data['annotations']:
#     segmentation = annotation['segmentation'][0]
#     polygon = [(segmentation[i], segmentation[i+1]) for i in range(0, len(segmentation), 2)]
#     ImageDraw.Draw(Image.fromarray(mask)).polygon(polygon, outline=1, fill=1)
#     mask = np.array(mask)
#     break

# # Save the mask as an image
# Image.fromarray(mask*255).save('mask.png')

In [ ]:
class DamagedDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    postfix = 'train' if train else 'val'
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, postfix), 
                                                           transform = transforms.Compose([
                                                               transforms.Resize(size),
                                                               transforms.ToTensor()
                                                               ]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root), 
                                                         transform = transforms.Compose([
                                                             transforms.Resize(size),
                                                             transforms.ToTensor()
                                                             ]))
  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)

Neural Net

In [ ]:
class DamageDetection(nn.Module):
  def __init__(self):
    super(DamageDetection, self).__init__()

    # input top left
    self.conv1 = nn.Conv2d(3,64,kernel_size=3, stride=1, padding=1)
    self.conv1b = nn.Conv2d(64,64,kernel_size=3, stride=1, padding=1)
    self.max1 = nn.MaxPool2d(kernel_size=2)

    self.conv2 = nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1)
    self.conv2b = nn.Conv2d(128,128,kernel_size=3, stride=1, padding=1)
    self.max2 = nn.MaxPool2d(kernel_size=2)

    self.conv3 = nn.Conv2d(128,256,kernel_size=3, stride=1, padding=1)
    self.conv3b = nn.Conv2d(256,256,kernel_size=3, stride=1, padding=1)
    self.max3 = nn.MaxPool2d(kernel_size=2)

    self.conv4 = nn.Conv2d(256,512,kernel_size=3, stride=1, padding=1)
    self.conv4b = nn.Conv2d(512,512,kernel_size=3, stride=1, padding=1)
    self.max4 = nn.MaxPool2d(kernel_size=2)
    
    # bottom
    self.conv5 = nn.Conv2d(512,1024,kernel_size=3, stride=1, padding=1)
    self.conv5b = nn.Conv2d(1024,1024,kernel_size=3, stride=1, padding=1)
    self.tran5 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
    # bottom
    
    self.conv6 = nn.Conv2d(1024,512,kernel_size=3, stride=1, padding=1)
    self.conv6b = nn.Conv2d(512,512,kernel_size=3, stride=1, padding=1)
    self.tran6 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)

    self.conv7 = nn.Conv2d(512,256,kernel_size=3, stride=1, padding=1)
    self.conv7b = nn.Conv2d(256,256,kernel_size=3, stride=1, padding=1)
    self.tran7 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)

    self.conv8 = nn.Conv2d(256,128,kernel_size=3, stride=1, padding=1)
    self.conv8b = nn.Conv2d(128,128,kernel_size=3, stride=1, padding=1)
    self.tran8 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

    self.conv9 = nn.Conv2d(128,64,kernel_size=3, stride=1, padding=1)
    self.conv9b = nn.Conv2d(64,64,kernel_size=3, stride=1, padding=1)
    # output top right
    self.conv10 = nn.Conv2d(64,2,kernel_size=1, stride=1, padding=0)


  def forward(self, input):
    # input top left
    convOut1a = self.conv1(input)
    relOut1a = F.relu(convOut1a)
    convOut1 = self.conv1b(relOut1a)
    relOut1 = F.relu(convOut1)     # concat with tranOut8
    maxOut1 = self.max1(relOut1)

    convOut2a = self.conv2(maxOut1)
    relOut2a = F.relu(convOut2a)
    convOut2 = self.conv2b(relOut2a)
    relOut2 = F.relu(convOut2)     # concat with tranOut7
    maxOut2 = self.max2(relOut2)

    convOut3a = self.conv3(maxOut2)
    relOut3a = F.relu(convOut3a)
    convOut3 = self.conv3b(relOut3a)
    relOut3 = F.relu(convOut3)     # concat with tranOut6
    maxOut3 = self.max3(relOut3)

    convOut4a = self.conv4(maxOut3)
    relOut4a = F.relu(convOut4a)
    convOut4 = self.conv4b(relOut4a)
    relOut4 = F.relu(convOut4)     # concat with tranOut5
    maxOut4 = self.max4(relOut4)

    # bottom
    convOut5a = self.conv5(maxOut4)
    relOut5a = F.relu(convOut5a)
    convOut5 = self.conv5b(relOut5a)
    relOut5 = F.relu(convOut5)
    tranOut5 = self.tran5(relOut5)
    # bottom

    concat6 = torch.cat((relOut4, tranOut5), dim=1)
    convOut6a = self.conv6(concat6)
    relOut6a = F.relu(convOut6a)
    convOut6 = self.conv6b(relOut6a)
    relOut6 = F.relu(convOut6)
    tranOut6 = self.tran6(relOut6)

    concat7 = torch.cat((relOut3, tranOut6), dim=1)
    convOut7a = self.conv7(concat7)
    relOut7a = F.relu(convOut7a)
    convOut7 = self.conv7b(relOut7a)
    relOut7 = F.relu(convOut7)
    tranOut7 = self.tran7(relOut7)

    concat8 = torch.cat((relOut2, tranOut7), dim=1)
    convOut8a = self.conv8(concat8)
    relOut8a = F.relu(convOut8a)
    convOut8 = self.conv8b(relOut8a)
    relOut8 = F.relu(convOut8)
    tranOut8 = self.tran8(relOut8)

    concat9 = torch.cat((relOut1, tranOut8), dim=1)
    convOut9a = self.conv9(concat9)
    relOut9a = F.relu(convOut9a)
    convOut9 = self.conv9b(relOut9a)
    relOut9 = F.relu(convOut9)

    convOut10 = self.conv10(relOut9)
    relOut10 = F.relu(convOut10)

    return relOut10

Initialize Classifier

Create datasets, dataloaders and neural network

In [ ]:
# TODO load masks so that they can load properly with the DamagedDataset class






# Initialize Datasets
train_dataset = DamagedDataset('/content/damage_coco', train=True)
val_dataset = DamagedDataset('/content/damage_coco/masks', train=False)

# Initialize Model
model = DamageDetection()
model = model.cuda()
 # Initialize Objective and Optimizer and other parameters
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

FileNotFoundError: ignored

In [ ]:
losses = []
accuracies = []
validations = []
val_accs = []
pre_img = []

Train

In [ ]:
def scope():
  try:
    #your code for calling dataset and dataloader
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, pin_memory=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=8, pin_memory=True, num_workers=2)
    gc.collect()
    print(torch.cuda.memory_allocated() / 1e9)
    
    val_acc = 0

    # Run your training and validation loop and collect stats
    for epoch in range(1):

      loop = tqdm(total=len(train_loader), position=0, leave=False)

      for batch, (x, y_truth) in enumerate(train_loader):
        x, y_truth = x.cuda(), y_truth.cuda()
        
        # Call your model, figure out loss and accuracy
        optimizer.zero_grad()
        y_hat = model(x)

        loss = objective(y_hat, y_truth.long())
        loss.backward()
        losses.append(loss.item())

        accuracy = (torch.softmax(y_hat, 1).argmax(1) == y_truth).float().mean()
        accuracies.append(accuracy.cpu())
        loop.set_description('epoch{}, loss:{:.4f}, accuracy:{:.3f} val_acc:{:.3f}'.format(epoch, loss.item(), accuracy, val_acc))
        loop.update(1)

        optimizer.step()

        if batch % 20 == 0:
          val = np.mean([objective(model(x.cuda()), y.cuda().long()).item() for x, y in val_loader])
          val_acc = np.mean([(model(x.cuda()).argmax(1) == y.cuda().long()).float().mean().item() for x, y in val_loader])
          validations.append((len(losses), val))
          val_accs.append((len(accuracies), val_acc))

      pred_img.append(model(val_dataset[172][0].unsqueeze(0).cuda()))

      loop.close()

  except:
    __ITB__()
    
scope()

Show Accuracy

In [ ]:
a, b = zip(*validations)

plt.plot(losses, label="Train")
plt.plot(a, b, label="Validation")
plt.xlabel("Steps")
plt.ylabel("Error")
plt.ylim(0, 1)
plt.title("Model Error")
plt.legend()
plt.show()

In [ ]:
a, b = zip(*val_accs)
print(f"Final Accuracy: {b[-1]}")

plt.plot(accuracies, label="Train")
plt.plot(a, b, label="Validation")
plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Model Accuracy")
plt.legend()
plt.show()

Testing

In [ ]:
import matplotlib.image as mpimg
plt.imshow(mpimg.imread('Training_Data/damage/damage_coco/test/0/1.jpg'))
plt.title("Original Image")

In [ ]:
for i, p in enumerate(pred_img):
  q = p.squeeze(0).argmax(0).cpu().detach().numpy()
  plt.imshow(q, cmap="gray")
  plt.title(f"Classification after Epoch {i}")